In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/pranav/intel-image-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [0]:
from tensorflow.keras.applications import vgg16

In [0]:
model = vgg16.VGG16(include_top=False, pooling='avg', input_shape = (150, 150, 3))

In [0]:
import numpy as np

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier


from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image as kimage

#First Cut

In [7]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [0]:
for layer in model.layers:
  layer.trainable = False

In [9]:
train_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
train_generator = train_processing.flow_from_directory('/content/data/seg_train/seg_train', 
                                                       batch_size=102,
                                                       target_size=(150,150), shuffle=False)

test_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
test_generator = test_processing.flow_from_directory('/content/data/seg_test/seg_test', 
                                                     batch_size=100,
                                                     target_size=(150,150), shuffle=False)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [10]:
%%time
for i in range(138): #round(14034/102)
  X, y = next(train_generator)
  if i == 0:
    X_train = model.predict(X)
    y_train = np.argmax(y, axis=1)
  else:
    X_train = np.concatenate((X_train, model.predict(X)))
    y_train = np.concatenate((y_train, np.argmax(y, axis=1)))
  if i%10==0 or i == 137:
    print(i, X_train.shape, y_train.shape)
np.save('/content/drive/My Drive/pranav/X_train_1', X_train)
np.save('/content/drive/My Drive/pranav/y_train_1', y_train)

0 (102, 512) (102,)
10 (1122, 512) (1122,)
20 (2142, 512) (2142,)
30 (3162, 512) (3162,)
40 (4182, 512) (4182,)
50 (5202, 512) (5202,)
60 (6222, 512) (6222,)
70 (7242, 512) (7242,)
80 (8262, 512) (8262,)
90 (9282, 512) (9282,)
100 (10302, 512) (10302,)
110 (11322, 512) (11322,)
120 (12342, 512) (12342,)
130 (13362, 512) (13362,)
137 (14034, 512) (14034,)
CPU times: user 31.1 s, sys: 7.87 s, total: 39 s
Wall time: 38.9 s


In [11]:
%%time
for i in range(30):
  X, y = next(test_generator)
  if i == 0:
    X_test = model.predict(X)
    y_test = np.argmax(y, axis=1)
  else:
    X_test = np.concatenate((X_test, model.predict(X)))
    y_test = np.concatenate((y_test, np.argmax(y, axis=1)))
  if i%10==0 or i == 29:
    print(i, X_test.shape, y_test.shape)
np.save('/content/drive/My Drive/pranav/X_test_1', X_test)
np.save('/content/drive/My Drive/pranav/y_test_1', y_test)

0 (100, 512) (100,)
10 (1100, 512) (1100,)
20 (2100, 512) (2100,)
29 (3000, 512) (3000,)
CPU times: user 6.33 s, sys: 1.54 s, total: 7.87 s
Wall time: 7.9 s


In [0]:
X_train = np.load('/content/drive/My Drive/pranav/X_train_1.npy')
y_train = np.load('/content/drive/My Drive/pranav/y_train_1.npy')
X_test  = np.load('/content/drive/My Drive/pranav/X_test_1.npy')
y_test  = np.load('/content/drive/My Drive/pranav/y_test_1.npy')

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [14]:
%%time
svclassifier = LinearSVC(dual=False, C=0.5, tol=1e-3).fit(X_train, y_train)
print(classification_report(y_train,svclassifier.predict(X_train),digits=4))
print(classification_report(y_val,svclassifier.predict(X_val),digits=4))

              precision    recall  f1-score   support

           0     0.9605    0.9610    0.9608      1720
           1     0.9978    0.9967    0.9973      1834
           2     0.9010    0.8873    0.8941      1907
           3     0.8957    0.9081    0.9019      2014
           4     0.9828    0.9828    0.9828      1803
           5     0.9641    0.9651    0.9646      1949

    accuracy                         0.9491     11227
   macro avg     0.9503    0.9502    0.9502     11227
weighted avg     0.9491    0.9491    0.9490     11227

              precision    recall  f1-score   support

           0     0.9087    0.9087    0.9087       471
           1     0.9709    0.9931    0.9819       437
           2     0.8388    0.8270    0.8328       497
           3     0.8563    0.8494    0.8528       498
           4     0.9293    0.9214    0.9254       471
           5     0.8995    0.9099    0.9047       433

    accuracy                         0.8992      2807
   macro avg     0.9006

In [15]:
print(classification_report(y_test,svclassifier.predict(X_test),digits=4))

              precision    recall  f1-score   support

           0     0.9141    0.8764    0.8949       437
           1     0.9752    0.9937    0.9843       474
           2     0.8514    0.7975    0.8235       553
           3     0.8139    0.8495    0.8313       525
           4     0.9304    0.9431    0.9367       510
           5     0.8932    0.9182    0.9055       501

    accuracy                         0.8940      3000
   macro avg     0.8963    0.8964    0.8960      3000
weighted avg     0.8939    0.8940    0.8936      3000



In [16]:
%%time
knnclassifier = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
print(classification_report(y_train,knnclassifier.predict(X_train),digits=4))
print(classification_report(y_val,knnclassifier.predict(X_val),digits=4))

              precision    recall  f1-score   support

           0     0.9306    0.9273    0.9289      1720
           1     0.9630    0.9940    0.9783      1834
           2     0.8860    0.9009    0.8934      1907
           3     0.9093    0.9012    0.9052      2014
           4     0.9791    0.9606    0.9698      1803
           5     0.9436    0.9277    0.9356      1949

    accuracy                         0.9344     11227
   macro avg     0.9353    0.9353    0.9352     11227
weighted avg     0.9346    0.9344    0.9344     11227

              precision    recall  f1-score   support

           0     0.8865    0.8790    0.8827       471
           1     0.9455    0.9931    0.9688       437
           2     0.7736    0.8249    0.7984       497
           3     0.8319    0.7851    0.8079       498
           4     0.9532    0.9087    0.9304       471
           5     0.8819    0.8799    0.8809       433

    accuracy                         0.8757      2807
   macro avg     0.8788

In [17]:
print(classification_report(y_test,knnclassifier.predict(X_test),digits=4))

              precision    recall  f1-score   support

           0     0.8721    0.8581    0.8651       437
           1     0.9497    0.9958    0.9722       474
           2     0.8127    0.8318    0.8222       553
           3     0.8327    0.8057    0.8190       525
           4     0.9450    0.9431    0.9441       510
           5     0.8918    0.8723    0.8819       501

    accuracy                         0.8827      3000
   macro avg     0.8840    0.8845    0.8841      3000
weighted avg     0.8822    0.8827    0.8823      3000



#Second Cut

In [18]:
x = GlobalAveragePooling2D()(model.layers[-6].output)
model_2 = Model(model.input, x)
model_2.compile(loss='mse',optimizer='sgd')
model_2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [19]:
%%time

train_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
train_generator = train_processing.flow_from_directory('/content/data/seg_train/seg_train', 
                                                       batch_size=102,
                                                       target_size=(150,150), shuffle=False)

test_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
test_generator = test_processing.flow_from_directory('/content/data/seg_test/seg_test', 
                                                     batch_size=100,
                                                     target_size=(150,150), shuffle=False)

for i in range(138): #round(14034/102)
  X, y = next(train_generator)
  if i == 0:
    X_train = model_2.predict(X)
    y_train = np.argmax(y, axis=1)
  else:
    X_train = np.concatenate((X_train, model_2.predict(X)))
    y_train = np.concatenate((y_train, np.argmax(y, axis=1)))
  if i%10==0 or i == 137:
    print(i, X_train.shape, y_train.shape)
np.save('/content/drive/My Drive/pranav/X_train_2', X_train)
np.save('/content/drive/My Drive/pranav/y_train_2', y_train)

for i in range(30):
  X, y = next(test_generator)
  if i == 0:
    X_test = model_2.predict(X)
    y_test = np.argmax(y, axis=1)
  else:
    X_test = np.concatenate((X_test, model_2.predict(X)))
    y_test = np.concatenate((y_test, np.argmax(y, axis=1)))
  if i%10==0 or i == 29:
    print(i, X_test.shape, y_test.shape)
np.save('/content/drive/My Drive/pranav/X_test_2', X_test)
np.save('/content/drive/My Drive/pranav/y_test_2', y_test)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
0 (102, 512) (102,)
10 (1122, 512) (1122,)
20 (2142, 512) (2142,)
30 (3162, 512) (3162,)
40 (4182, 512) (4182,)
50 (5202, 512) (5202,)
60 (6222, 512) (6222,)
70 (7242, 512) (7242,)
80 (8262, 512) (8262,)
90 (9282, 512) (9282,)
100 (10302, 512) (10302,)
110 (11322, 512) (11322,)
120 (12342, 512) (12342,)
130 (13362, 512) (13362,)
137 (14034, 512) (14034,)
0 (100, 512) (100,)
10 (1100, 512) (1100,)
20 (2100, 512) (2100,)
29 (3000, 512) (3000,)
CPU times: user 43.6 s, sys: 8.15 s, total: 51.7 s
Wall time: 49.1 s


In [0]:
X_train = np.load('/content/drive/My Drive/pranav/X_train_2.npy')
y_train = np.load('/content/drive/My Drive/pranav/y_train_2.npy')
X_test  = np.load('/content/drive/My Drive/pranav/X_test_2.npy')
y_test  = np.load('/content/drive/My Drive/pranav/y_test_2.npy')

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [22]:
%%time
svclassifier = LinearSVC(dual=False, C=1, tol=1e-3).fit(X_train, y_train)
print(classification_report(y_train,svclassifier.predict(X_train),digits=4))
print(classification_report(y_val,svclassifier.predict(X_val),digits=4))

              precision    recall  f1-score   support

           0     0.9753    0.9731    0.9742      1748
           1     0.9989    1.0000    0.9994      1802
           2     0.9203    0.8957    0.9079      1947
           3     0.9102    0.9241    0.9171      2028
           4     0.9796    0.9856    0.9826      1807
           5     0.9738    0.9799    0.9769      1895

    accuracy                         0.9583     11227
   macro avg     0.9597    0.9597    0.9597     11227
weighted avg     0.9582    0.9583    0.9582     11227

              precision    recall  f1-score   support

           0     0.9076    0.8871    0.8973       443
           1     0.9872    0.9851    0.9861       469
           2     0.8653    0.8578    0.8615       457
           3     0.8492    0.8843    0.8664       484
           4     0.9356    0.9336    0.9346       467
           5     0.9068    0.8994    0.9031       487

    accuracy                         0.9081      2807
   macro avg     0.9086

In [23]:
print(classification_report(y_test,svclassifier.predict(X_test),digits=4))

              precision    recall  f1-score   support

           0     0.9183    0.8741    0.8957       437
           1     0.9833    0.9916    0.9874       474
           2     0.8696    0.8318    0.8503       553
           3     0.8447    0.8705    0.8574       525
           4     0.9070    0.9373    0.9219       510
           5     0.9096    0.9242    0.9168       501

    accuracy                         0.9033      3000
   macro avg     0.9054    0.9049    0.9049      3000
weighted avg     0.9033    0.9033    0.9031      3000



In [24]:
%%time
knnclassifier = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
print(classification_report(y_train,knnclassifier.predict(X_train),digits=4))
print(classification_report(y_val,knnclassifier.predict(X_val),digits=4))

              precision    recall  f1-score   support

           0     0.9674    0.8839    0.9238      1748
           1     0.9824    0.9939    0.9881      1802
           2     0.8628    0.9301    0.8952      1947
           3     0.9249    0.8866    0.9053      2028
           4     0.9749    0.9225    0.9480      1807
           5     0.8973    0.9726    0.9334      1895

    accuracy                         0.9312     11227
   macro avg     0.9350    0.9316    0.9323     11227
weighted avg     0.9334    0.9312    0.9313     11227

              precision    recall  f1-score   support

           0     0.9475    0.7743    0.8522       443
           1     0.9767    0.9829    0.9798       469
           2     0.7607    0.8906    0.8206       457
           3     0.8467    0.8099    0.8279       484
           4     0.9685    0.8565    0.9091       467
           5     0.8238    0.9507    0.8827       487

    accuracy                         0.8785      2807
   macro avg     0.8873

In [25]:
print(classification_report(y_test,knnclassifier.predict(X_test),digits=4))

              precision    recall  f1-score   support

           0     0.9391    0.7757    0.8496       437
           1     0.9793    0.9979    0.9885       474
           2     0.7767    0.8680    0.8198       553
           3     0.8310    0.7867    0.8082       525
           4     0.9486    0.8686    0.9069       510
           5     0.8293    0.9501    0.8856       501

    accuracy                         0.8747      3000
   macro avg     0.8840    0.8745    0.8764      3000
weighted avg     0.8799    0.8747    0.8746      3000



#Third Cut

In [26]:
x = GlobalAveragePooling2D()(model.layers[-10].output)
model_3 = Model(model.input, x)
model_3.compile(loss='mse',optimizer='sgd') #è indifferente
model_3.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [27]:
%%time

train_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
train_generator = train_processing.flow_from_directory('/content/data/seg_train/seg_train', 
                                                       batch_size=102,
                                                       target_size=(150,150), shuffle=False)

test_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
test_generator = test_processing.flow_from_directory('/content/data/seg_test/seg_test', 
                                                     batch_size=100,
                                                     target_size=(150,150), shuffle=False)

for i in range(138): #round(14034/102)
  X, y = next(train_generator)
  if i == 0:
    X_train = model_3.predict(X)
    y_train = np.argmax(y, axis=1)
  else:
    X_train = np.concatenate((X_train, model_3.predict(X)))
    y_train = np.concatenate((y_train, np.argmax(y, axis=1)))
  if i%10==0 or i == 137:
    print(i, X_train.shape, y_train.shape)
np.save('/content/drive/My Drive/pranav/X_train_3', X_train)
np.save('/content/drive/My Drive/pranav/y_train_3', y_train)

for i in range(30):
  X, y = next(test_generator)
  if i == 0:
    X_test = model_3.predict(X)
    y_test = np.argmax(y, axis=1)
  else:
    X_test = np.concatenate((X_test, model_3.predict(X)))
    y_test = np.concatenate((y_test, np.argmax(y, axis=1)))
  if i%10==0 or i == 29:
    print(i, X_test.shape, y_test.shape)
np.save('/content/drive/My Drive/pranav/X_test_3', X_test)
np.save('/content/drive/My Drive/pranav/y_test_3', y_test)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
0 (102, 256) (102,)
10 (1122, 256) (1122,)
20 (2142, 256) (2142,)
30 (3162, 256) (3162,)
40 (4182, 256) (4182,)
50 (5202, 256) (5202,)
60 (6222, 256) (6222,)
70 (7242, 256) (7242,)
80 (8262, 256) (8262,)
90 (9282, 256) (9282,)
100 (10302, 256) (10302,)
110 (11322, 256) (11322,)
120 (12342, 256) (12342,)
130 (13362, 256) (13362,)
137 (14034, 256) (14034,)
0 (100, 256) (100,)
10 (1100, 256) (1100,)
20 (2100, 256) (2100,)
29 (3000, 256) (3000,)
CPU times: user 40.8 s, sys: 5.88 s, total: 46.7 s
Wall time: 44.1 s


In [0]:
X_train = np.load('/content/drive/My Drive/pranav/X_train_3.npy')
y_train = np.load('/content/drive/My Drive/pranav/y_train_3.npy')
X_test  = np.load('/content/drive/My Drive/pranav/X_test_3.npy')
y_test  = np.load('/content/drive/My Drive/pranav/y_test_3.npy')

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [29]:
%%time
svclassifier = LinearSVC(dual=False, C=0.5, tol=1e-3).fit(X_train, y_train)
print(classification_report(y_train,svclassifier.predict(X_train),digits=4))
print(classification_report(y_val,svclassifier.predict(X_val),digits=4))

              precision    recall  f1-score   support

           0     0.9321    0.9215    0.9267      1757
           1     0.9940    0.9918    0.9929      1828
           2     0.8738    0.8566    0.8651      1932
           3     0.8624    0.8763    0.8693      1988
           4     0.9292    0.9333    0.9313      1800
           5     0.9285    0.9391    0.9338      1922

    accuracy                         0.9187     11227
   macro avg     0.9200    0.9198    0.9198     11227
weighted avg     0.9187    0.9187    0.9187     11227

              precision    recall  f1-score   support

           0     0.9011    0.9032    0.9022       434
           1     0.9776    0.9865    0.9820       443
           2     0.8302    0.8390    0.8346       472
           3     0.8600    0.8206    0.8398       524
           4     0.9040    0.9135    0.9087       474
           5     0.9019    0.9196    0.9107       460

    accuracy                         0.8945      2807
   macro avg     0.8958

In [30]:
print(classification_report(y_test,svclassifier.predict(X_test),digits=4))

              precision    recall  f1-score   support

           0     0.9076    0.8993    0.9034       437
           1     0.9874    0.9895    0.9884       474
           2     0.8566    0.8318    0.8440       553
           3     0.8356    0.8324    0.8340       525
           4     0.8945    0.9314    0.9126       510
           5     0.9102    0.9102    0.9102       501

    accuracy                         0.8967      3000
   macro avg     0.8986    0.8991    0.8988      3000
weighted avg     0.8964    0.8967    0.8964      3000



#Forth Cut

In [31]:
x = GlobalAveragePooling2D()(model.layers[-14].output)
model_4 = Model(model.input, x)
model_4.compile(loss='mse',optimizer='sgd')
model_4.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [32]:
%%time

train_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
train_generator = train_processing.flow_from_directory('/content/data/seg_train/seg_train', 
                                                       batch_size=102,
                                                       target_size=(150,150), shuffle=False)

test_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
test_generator = test_processing.flow_from_directory('/content/data/seg_test/seg_test', 
                                                     batch_size=100,
                                                     target_size=(150,150), shuffle=False)

for i in range(138): #round(14034/102)
  X, y = next(train_generator)
  if i == 0:
    X_train = model_4.predict(X)
    y_train = np.argmax(y, axis=1)
  else:
    X_train = np.concatenate((X_train, model_4.predict(X)))
    y_train = np.concatenate((y_train, np.argmax(y, axis=1)))
  if i%10==0 or i == 137:
    print(i, X_train.shape, y_train.shape)
np.save('/content/drive/My Drive/pranav/X_train_4', X_train)
np.save('/content/drive/My Drive/pranav/y_train_4', y_train)

for i in range(30):
  X, y = next(test_generator)
  if i == 0:
    X_test = model_4.predict(X)
    y_test = np.argmax(y, axis=1)
  else:
    X_test = np.concatenate((X_test, model_4.predict(X)))
    y_test = np.concatenate((y_test, np.argmax(y, axis=1)))
  if i%10==0 or i == 29:
    print(i, X_test.shape, y_test.shape)
np.save('/content/drive/My Drive/pranav/X_test_4', X_test)
np.save('/content/drive/My Drive/pranav/y_test_4', y_test)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
0 (102, 128) (102,)
10 (1122, 128) (1122,)
20 (2142, 128) (2142,)
30 (3162, 128) (3162,)
40 (4182, 128) (4182,)
50 (5202, 128) (5202,)
60 (6222, 128) (6222,)
70 (7242, 128) (7242,)
80 (8262, 128) (8262,)
90 (9282, 128) (9282,)
100 (10302, 128) (10302,)
110 (11322, 128) (11322,)
120 (12342, 128) (12342,)
130 (13362, 128) (13362,)
137 (14034, 128) (14034,)
0 (100, 128) (100,)
10 (1100, 128) (1100,)
20 (2100, 128) (2100,)
29 (3000, 128) (3000,)
CPU times: user 38.4 s, sys: 4.3 s, total: 42.7 s
Wall time: 40 s


In [0]:
X_train = np.load('/content/drive/My Drive/pranav/X_train_4.npy')
y_train = np.load('/content/drive/My Drive/pranav/y_train_4.npy')
X_test  = np.load('/content/drive/My Drive/pranav/X_test_4.npy')
y_test  = np.load('/content/drive/My Drive/pranav/y_test_4.npy')

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [35]:
%%time
svclassifier = LinearSVC(dual=False, C=0.1, tol=1e-3).fit(X_train, y_train)
print(classification_report(y_train,svclassifier.predict(X_train),digits=4))
print(classification_report(y_val,svclassifier.predict(X_val),digits=4))

              precision    recall  f1-score   support

           0     0.8402    0.8074    0.8235      1745
           1     0.9720    0.9736    0.9728      1818
           2     0.8165    0.7937    0.8049      1929
           3     0.7812    0.8109    0.7958      2004
           4     0.8488    0.8431    0.8459      1817
           5     0.8314    0.8553    0.8432      1914

    accuracy                         0.8465     11227
   macro avg     0.8484    0.8473    0.8477     11227
weighted avg     0.8468    0.8465    0.8465     11227

              precision    recall  f1-score   support

           0     0.8169    0.7601    0.7875       446
           1     0.9502    0.9691    0.9596       453
           2     0.8176    0.8021    0.8098       475
           3     0.7895    0.8268    0.8077       508
           4     0.8575    0.8556    0.8565       457
           5     0.7878    0.8013    0.7945       468

    accuracy                         0.8354      2807
   macro avg     0.8366

In [36]:
print(classification_report(y_test,svclassifier.predict(X_test),digits=4))

              precision    recall  f1-score   support

           0     0.8264    0.7735    0.7991       437
           1     0.9579    0.9599    0.9589       474
           2     0.8019    0.7758    0.7886       553
           3     0.7541    0.7943    0.7737       525
           4     0.8268    0.8333    0.8301       510
           5     0.8249    0.8463    0.8355       501

    accuracy                         0.8293      3000
   macro avg     0.8320    0.8305    0.8310      3000
weighted avg     0.8298    0.8293    0.8293      3000

